# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

### 3) Configuration 불러오기

In [2]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
# pprint(loaded_config)


data_path = loaded_config['general']['data_path']
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))

## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [3]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token
    
    @staticmethod    
    def preprocess_data(df):
        df = df.copy()
        # fname 정리
        df['fname'] = df['fname'].str.strip()
        
        # summary 정리
        if 'summary' in df.columns:
            df['summary'] = df['summary'].str.strip()
            df['summary'] = df['summary'].str.replace(r'\s+', ' ', regex=True)
            df['summary'] = df['summary'].str.strip('"').str.strip("'")
        
        # dialogue 정리
        if 'dialogue' in df.columns:
            df['dialogue'] = df['dialogue'].str.strip()
            df['dialogue'] = df['dialogue'].str.replace(r'\s+', ' ', regex=True)
            df['dialogue'] = df['dialogue'].str.strip('"').str.strip("'")
        
        return df
                

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            train_df = Preprocess.preprocess_data(train_df)
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            test_df = Preprocess.preprocess_data(test_df)
            return test_df
        
    def preprocess_data(df):
        df = df.copy()
        # fname 정리
        df['fname'] = df['fname'].str.strip()
        
        # summary 정리
        if 'summary' in df.columns:
            df['summary'] = df['summary'].str.strip()
            df['summary'] = df['summary'].str.replace(r'\s+', ' ', regex=True)
            df['summary'] = df['summary'].str.strip('"').str.strip("'")
        
        # dialogue 정리
        if 'dialogue' in df.columns:
            df['dialogue'] = df['dialogue'].str.strip()
            df['dialogue'] = df['dialogue'].str.replace(r'\s+', ' ', regex=True)
        
        return df
            

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset,is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
            decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [4]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [5]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [6]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [7]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                eval_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                metric_for_best_model="rouge-l",
                greater_is_better=True,  
                report_to=[],
                # report_to=config['training']['report_to'], # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    # wandb.init(
    #     entity=config['wandb']['entity'],
    #     project=config['wandb']['project'],
    #     name=config['wandb']['name'],
    #     settings=wandb.Settings(start_method="thread", init_timeout=300)
    # )

    # # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    # os.environ["WANDB_LOG_MODEL"]="false"
    # os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [8]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    bart_config.tie_word_embeddings = True
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    # print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [9]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    # wandb.finish()

In [10]:
import torch
import gc

# GPU 메모리 완전 초기화
torch.cuda.empty_cache()
gc.collect()

# 모든 CUDA 텐서 삭제
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) and obj.is_cuda:
            del obj
    except:
        pass

gc.collect()
torch.cuda.empty_cache()

# 1. 현재 할당된 메모리 (PyTorch가 사용 중)
allocated = torch.cuda.memory_allocated() / 1024**3
print(f"Allocated: {allocated:.2f} GB")  # 사용 중인 메모리

# 2. 예약된 메모리 (PyTorch가 확보해둔 메모리)
reserved = torch.cuda.memory_reserved() / 1024**3
print(f"Reserved: {reserved:.2f} GB")    # 캐시 포함

# 3. 전체 GPU 메모리
total = torch.cuda.get_device_properties(0).total_memory / 1024**3
print(f"Total: {total:.2f} GB")          # GPU 전체 용량

# 4. 사용 가능한 메모리 (대략)
free = total - reserved
print(f"Free: {free:.2f} GB")

/opt/conda/envs/nlp/lib/python3.10/site-packages/torch/__init__.py:1125: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)


Allocated: 0.00 GB
Reserved: 0.00 GB
Total: 23.69 GB
Free: 23.69 GB


In [13]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.8.0+cu128
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


---------- Load tokenizer & model complete ----------
---------- tokenizer special tokens :  {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'additional_special_tokens': ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']} ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
train_data:
 #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? #Person2#: 건강검진을 받으려고 왔어요. #Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. #Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. #Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. #Person2#: 알겠습니다. #Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? #Person2#: 네. #Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. #Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요. #Person1#: 음, 도움 될만한 수업과 약물들이 있습니다. 가시기 전에 더 정보를 드릴게요.

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


---------- Make dataset complete ----------
---------- Make training arguments ----------
---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,5.453700,2.625999,0.185879,0.027640,0.180173
2,1.470700,0.777520,0.240320,0.043300,0.232167
3,0.689200,0.603174,0.339071,0.106703,0.318066
4,0.591200,0.586965,0.357202,0.122394,0.336559
5,0.550800,0.578037,0.362526,0.129147,0.342459
6,0.521400,0.574777,0.363844,0.132843,0.343152
7,0.495100,0.572173,0.368285,0.139077,0.349272
8,0.476500,0.573477,0.364492,0.129945,0.339190
9,0.459200,0.573735,0.368365,0.134475,0.345821
10,0.443900,0.576410,0.373521,0.140685,0.350851


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:       는 천식 검사를 위해 폐 전문의에게 가보시는 것을 추천합니다.                                                                                
GOLD: #Person2# 는 숨쉬기 어려워합니다. 의사는 #Person2# 에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   농구하러 다리가 아파서 운동하러 가자는   의 제안에 대해   는  금요일에 운동하자고 말합니다.                                                                         
GOLD: #Person1# 는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.                                                                       
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   무릎

## 4. 모델 추론하기

In [19]:
# 새롭게 parameter 설정한 config 파일을 다시 한번 불러오기!
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "/workspace/NLP_Dialogue_Summarization/output/model/checkpoint-4500"

- test data를 사용하여 모델의 성능을 확인합니다.

In [15]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [16]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [ ]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]
        
    # ✅ 공백/특수문자 추가 정리
    cleaned_summary = []
    for s in preprocessed_summary:
        s = s.strip()                         # 앞뒤 공백 제거
        s = re.sub(r'\s+', ' ', s)            # 여러 공백 → 하나
        s = s.strip('"').strip("'")           # 따옴표 제거
        cleaned_summary.append(s)    

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : cleaned_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [23]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.8.0+cu128
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. #Person2#: 네, 말씀하세요... #Person1#: 이걸 오늘 오후까지 모든 직원들에게 사내 메모로 보내야 해요. 준비됐나요? #Person2#: 네, 말씀하세요. #Person1#: 모든 직원에게 알립니다... 즉시 발효되어 모든 사내 통신은 이메일과 공식 메모로만 제한됩니다. 근무 시간 동안 즉시 메시지 프로그램 사용은 금지됩니다. #Person2#: 이 정책이 사내 통신에만 적용되나요, 아니면 외부 통신에도 해당되나요? #Person1#: 이는 모든 통신에 적용됩니다. 사무실 내 직원 간 통신 뿐만 아니라 외부 통신도 해당됩니다. #Person2#: 하지만 많은 직원들이 고객과 소통하려고 즉시 메시지를 사용합니다. #Person1#: 통신 방법을 바꿔야 할 것입니다. 이 사무실에서는 즉시 메시지를 사용하는 것을 원하지 않습니다. 너무 많은 시간이 낭비됩니다! 이제 계속해서 메모를 작성해 주세요. 어디까지 했죠? #Person2#: 내외부 통신에 적용됩니다. #Person1#: 네. 즉시 메시지를 계속 사용하면 경고 후 시정 조치가 이루어지며, 두 번째 위반 시 해고될 수 있습니다. 이번 정책에 관한 질문은 부서장에게 문의하세요. #Person2#: 그게 다인가요? #Person1#: 네. 오늘 오후 4시까지 이 메모를 작성하고 배포해주세요.
----------------------------------------------------------------------------------------

100%|██████████| 16/16 [00:58<00:00,  3.63s/it]


In [24]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,#Person1# 은 Ms. Dawson에게 사내 메모를 작성하고 배포할 것을 요청...
1,test_1,#Person1# 과 #Person2# 는 교통체증으로 인해 출퇴근길에 대해 논의하...
2,test_2,#Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거 중이며 ...
3,test_3,#Person1# 은 Brian의 생일을 축하하며 파티에서 함께 춤을 추자고 제안합니다.
4,test_4,#Person1# 과 #Person2# 는 올림픽 공원의 크기와 시설에 대해 이야기...
...,...,...
494,test_495,"Jack은 Charlie에게 새로운 비디오 게임을 제안하고, C는 숙제를 먼저 끝내..."
495,test_496,#Person2# 는 시골 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일하게...
496,test_497,"Alice는 #Person1# 에게 세탁기에 비누가 들어 있지 않다고 설명하고, 세..."
497,test_498,"Steve는 Matthew에게 임대를 갱신하고 싶지 않다고 말하며, 이웃인 Mrs...."


## submission csv 파일에서 조사 붙이는 작업 진행!!

In [ ]:
import pandas as pd
import re

# 1) 파일 로드
df = pd.read_csv("/workspace/NLP_Dialogue_Summarization/output/submission/3.baseline_with_pre_post.csv", encoding="utf-8")

# 2) 요약문 컬럼명
TEXT_COL = "summary"

# 3) 붙일 조사(확장판)
# - 주제/보격/목적/접속/보조/격조사 등 흔히 등장하는 형태 위주
particles = [
    # 주제/보격/목적/소유
    "은","는","이","가","을","를","의","께서","에서의",
    # 접속/공동
    "과","와","랑","이랑","하고","및",
    # 부사격/여러 격
    "에","에서","에게","에게서","한테","한테서","께","대해","대해서",
    # 방향/수단/자격
    "으로","로","으로서","로서","으로써","로써",
    # 보조/인용/서술 연결 (띄어쓰기 오류 교정 용도)
    "라고","이라며","이라서","라서","이어서","여서",
    # 보편 빈도 조사
    "도","만","뿐","부터","까지","마다","조차","마저","조차도","보다","만큼","처럼","같이","밖에"
]

# 4) 정규식 준비
#  - 아이디어: "앞에 공백이 있고, 뒤에는 조사" 인 공백을 없애기
#  - lookbehind로 "앞이 공백이 아닌 문자"일 때만 동작 → 문장 첫머리 등은 보호
particles_regex = r"(?:{})".format("|".join(map(re.escape, particles)))
pattern = re.compile(r"(?<=[^\s])\s+(?={})".format(particles_regex))

def join_particles(text: str) -> str:
    if pd.isna(text):
        return text
    # 공백 → '' (삭제) : 공백만 지우고 조사는 유지되니 자동으로 앞 단어에 붙음
    return pattern.sub("", str(text))

# 5) 적용
df[TEXT_COL] = df[TEXT_COL].apply(join_particles)

# 6) 저장
df.to_csv("/workspace/NLP_Dialogue_Summarization/output/submission/5-1.attach_particle.csv", index=False, encoding="utf-8-sig")
print("✅ 완료: 5.attach_particle.csv")


✅ 완료: 5.attach_particle.csv


In [3]:
import pandas as pd
import re

# 1) 샘플 CSV 불러오기
df = pd.read_csv("/workspace/NLP_Dialogue_Summarization/output/submission/3.baseline_with_pre_post.csv", encoding="utf-8")

# 2) 요약문 컬럼명
TEXT_COL = "summary"

# 3) 조사 리스트 (확장 가능)
particles = [
    "은","는","이","가","을","를","의","께서","에서의",
    "과","와","랑","이랑","하고","및",
    "에","에서","에게","에게서","한테","한테서","께","대해","대해서",
    "으로","로","으로서","로서","으로써","로써",
    "라고","이라며","이라서","라서","이어서","여서",
    "도","만","뿐","부터","까지","마다","조차","마저","보다","만큼","처럼","같이","밖에"
]

# 4) 정규식 준비
particles_regex = r"(?:{})".format("|".join(map(re.escape, particles)))
pattern = re.compile(r"(?<=[^\s])\s+(?={})".format(particles_regex))

def join_particles(text: str) -> str:
    if pd.isna(text):
        return text
    return pattern.sub("", str(text))

# 5) 원본과 변환 비교
changed_mask = df[TEXT_COL].apply(lambda x: join_particles(x)) != df[TEXT_COL]
changed_rows = df.loc[changed_mask, "fname"]

print("조사 붙이기 적용된 fname 개수:", changed_rows.shape[0])
print("적용된 fname 목록:\n", changed_rows.tolist())


조사 붙이기 적용된 fname 개수: 475
적용된 fname 목록:
 ['test_0', 'test_1', 'test_2', 'test_3', 'test_4', 'test_5', 'test_6', 'test_7', 'test_8', 'test_9', 'test_10', 'test_11', 'test_12', 'test_14', 'test_15', 'test_16', 'test_17', 'test_18', 'test_19', 'test_21', 'test_22', 'test_23', 'test_24', 'test_25', 'test_26', 'test_27', 'test_28', 'test_29', 'test_30', 'test_32', 'test_34', 'test_35', 'test_36', 'test_37', 'test_38', 'test_39', 'test_40', 'test_41', 'test_42', 'test_43', 'test_44', 'test_46', 'test_47', 'test_48', 'test_50', 'test_51', 'test_52', 'test_53', 'test_54', 'test_55', 'test_56', 'test_57', 'test_58', 'test_59', 'test_60', 'test_61', 'test_62', 'test_63', 'test_64', 'test_65', 'test_66', 'test_67', 'test_68', 'test_70', 'test_71', 'test_72', 'test_73', 'test_74', 'test_75', 'test_76', 'test_77', 'test_78', 'test_79', 'test_80', 'test_81', 'test_82', 'test_83', 'test_84', 'test_85', 'test_86', 'test_87', 'test_88', 'test_89', 'test_90', 'test_91', 'test_92', 'test_93', 'test_94', '